In [5]:
import nltk
from rouge_score import rouge_scorer
from rouge import Rouge 
import os 
import pickle
import glob

In [6]:
path = "/Users/derriesusan/Documents/Coursework/DS5500/AMICorpusXML-master/data"

In [7]:
def get_dict(datapath,typ):
    os.chdir(datapath)
    filenames = glob.glob('*.txt')
    textdict = {}
    extn = ''
    if typ == 'summary':
        extn = '.extsumm.txt'
    elif typ == 'transcript':
        extn = '.transcript.txt'
    for f in filenames:
        with open(f,'r') as file:
            if f in textdict.keys():
                continue
            key = f[:-len(extn)]
            val = file.read()
            textdict[key] = val
    return textdict

In [8]:
trans_dict = get_dict(path+'/ami-transcripts','transcript')
summ_dict = get_dict(path+'/ami-summary/extractive','summary')
result = {}
for key in (trans_dict.keys() & summ_dict.keys()):
    if key in trans_dict: result.setdefault(key, []).append(trans_dict[key])
    if key in summ_dict: result.setdefault(key, []).append(summ_dict[key])

In [9]:
stop_words= ['uh','cool','right','cool','um','oh','mm','mm-hmm','huh','th','tu','yeah','yep','er','okay','oops',"'kay",'hmm','uh-huh','uh,','um,','oh,','mm,','mm-hmm,','huh,','th,','yeah,','yep,','er,','okay,',"kay,",'hmm,','uh-huh,','well,','well']

clean_dict = result.copy()
transcripts_list_clean=[]
for i in result.keys():
    tokenized_trans = (nltk.word_tokenize(result[i][0].lower()))
    tokenized_summ = (nltk.word_tokenize(result[i][1].lower()))
    cleaned_trans = [w for w in tokenized_trans if not w in stop_words]
    cleaned_summ = [w for w in tokenized_summ if not w in stop_words]
    clean_dict[i][0] = " ".join(cleaned_trans)
    clean_dict[i][1] = " ".join(cleaned_summ)


In [10]:
def remove_punc(resdict):
    values = [0,1]
    for val in range(len(values)):
        for i in resdict.keys():
            split_sentence = resdict[i][val].split(".")
            while "" in split_sentence:
                split_sentence.remove("")
            while " " in split_sentence:
                split_sentence.remove(" ")
            while " , " in split_sentence:
                split_sentence.remove(" , ")
            resdict[i][val]=".".join(split_sentence)
        
        for i in resdict.keys():  
            split_sentence = resdict[i][val].split(",")
            while "" in split_sentence:
                split_sentence.remove("")
            while " . " in split_sentence:
                split_sentence.remove(" . ")
            while " " in split_sentence:
                split_sentence.remove(" ")
            resdict[i][val]=",".join(split_sentence)
        
        for i in resdict.keys():   
            split_sentence = resdict[i][val].split(". ,")
            while "" in split_sentence:
                split_sentence.remove("")
            resdict[i][val] = ".".join(split_sentence)
        
        for i in resdict.keys():    
            split_sentence = resdict[i][val].split("? .")
            while "" in split_sentence:
                split_sentence.remove("")
            resdict[i][val]="?".join(split_sentence)
            
#         resdict[i][val] = nltk.sent_tokenize(resdict[i][val])

    return resdict

In [7]:
def addsep_token(resdict):
    values = [0,1]
    for val in values:
        for i in resdict.keys():
            split_sentence = resdict[i][val].split(".")
            while "" in split_sentence:
                split_sentence.remove("")
            while " " in split_sentence:
                split_sentence.remove(" ")
            resdict[i][val] = "<S_SEP>".join(split_sentence)
            
        for i in resdict.keys():
            split_sentence = resdict[i][val].split(" ? ")
            while "" in split_sentence:
                split_sentence.remove("")
            while " " in split_sentence:
                split_sentence.remove(" ")
            resdict[i][val] = "? <S_SEP> ".join(split_sentence)
    return resdict

In [11]:
new_dict = remove_punc(clean_dict)

In [17]:
trans_dict['ES2006b']

"Mm. You said uh targ target groups, what does that mean? Uh okay, 'kay. So are Okay. Alright. I can go first, yeah. Right. Um so f from the Right sure. Uh okay. So n uh with uh with regard to the uh working design of this uh uh remote control uh I've identified um a few basic uh components of the remote and uh se uh from the design, functional design perspective um w I c we can now uh know wha what exactly the components are and how how they work together with each other. So this is the method that uh I'll mostly be following in my um in my uh role. Um the identification of the components, uh and uh since since I'm dealing only with the technical aspects, I would need feedback from the marketing person uh and uh from the user interface person. Uh we'll then integrate this into the product design at a technical level and uh basically update and come up with a new design, so it's a cyclical process. Okay, so these were the basic findings from today. The last three bullets have been inte

In [16]:
new_dict['ES2006b'][0]

" you said targ target groups , what does that mean ? , . so are . alright . i can go first , . so f from the sure . so n with with regard to the working design of this remote control i 've identified a few basic components of the remote and se from the design , functional design perspective w i c we can now know wha what exactly the components are and how how they work together with each other . so this is the method that i 'll mostly be following in my in my role . the identification of the components , and since since i 'm dealing only with the technical aspects , i would need feedback from the marketing person and from the user interface person . we 'll then integrate this into the product design at a technical level and basically update and come up with a new design , so it 's a cyclical process . so these were the basic findings from today . the last three bullets have been integrated from the last minute email . i just quickly jotted them down . so basically the as i told you th

In [11]:
transcript_sent_tokenized = []
summary_sent_tokenized = []

for i in new_dict.keys():
    transcript_sent_tokenized.append(nltk.sent_tokenize(new_dict[i][0]))
    summary_sent_tokenized.append(nltk.sent_tokenize(new_dict[i][1]))
    
  

In [12]:
labels = []
# scorer = rouge_scorer.RougeScorer('rougeL', use_stemmer=True)
rouge = Rouge()

for i in range(0,len(transcript_sent_tokenized)):
    label = [0]*len(transcript_sent_tokenized[i])
    for each_sent in summary_sent_tokenized[i]:
        scores = list()
        for each_line in transcript_sent_tokenized[i]:
            if each_sent!='...' and each_sent!='.' and each_line!='...'and each_line!='.' and each_line!='....' and each_sent!='....' and each_line!='.....' and each_line!='.....':
                score = rouge.get_scores(each_line, each_sent)
                scores.append(score[0].get('rouge-l').get('f'))
            
        
#         scores.append(scorer.score(x,j))
        if each_sent!='...' and each_sent!='.' and each_line!='...'and each_line!='.' and each_line!='....' and each_sent!='....' and each_line!='.....' and each_line!='.....':
            pos= scores.index(max(scores))
            label[pos] = 1
        
            
    labels.append(label) 

In [15]:
dataset_dict = []
for i in range(0,len(transcript_sent_tokenized)):
    temp = {'transcript': transcript_sent_tokenized[i], 'summary': summary_sent_tokenized[i], 'labels':labels[i] }
    dataset_dict.append(temp)

In [18]:
with open('/Users/derriesusan/Documents/Coursework/DS5500/final_data.pickle', 'wb') as messy_data:
    pickle.dump(dataset_dict, messy_data)


In [13]:
labels = []
# scorer = rouge_scorer.RougeScorer('rougeL', use_stemmer=True)
rouge = Rouge()

for i in new_dict.keys():
    label = [0]*len(new_dict[i][0])
    for each_sent in new_dict[i][1]:
        scores = list()
        for each_line in new_dict[i][0]:
            if each_sent!='...' and each_sent!='.' and each_line!='...'and each_line!='.' and each_line!='....' and each_sent!='....' and each_line!='.....' and each_line!='.....':
                score = rouge.get_scores(each_line, each_sent)
                scores.append(score[0].get('rouge-l').get('f'))

        if each_sent!='...' and each_sent!='.' and each_line!='...'and each_line!='.' and each_line!='....' and each_sent!='....' and each_line!='.....' and each_line!='.....':
            pos= scores.index(max(scores))
            label[pos] = 1
        
            
    new_dict[i].append(label)  

a
s
 
y
o
u
 
a
l
l
 
k
n
o
w
 
w
e
 
'
r
e
 
h
e
r
e
 
t
o
 
c
r
e
a
t
e
 
a
 
b
r
a
n
d
 
n
e
w
 
f
a
n
t
a
s
t
i
c
 
r
e
m
o
t
e
 
.
 
i
 
'
m
 
n
i
c
k


KeyboardInterrupt: 

In [ ]:
for each_sent in new_dict['ES2002a'][0]:
    print(each_sent)

In [ ]:
final_dict = addsep_token(new_dict)

In [ ]:
os.chdir("../../..")
import more-itertools
outF = open("./data/training.article", "w")
outF.write(more_itertools.take(103, final_dict.items()))
outF.close()
outF = open("./data/training.label", "w")
for i in range(0,103):
    outF.write(str(labels[i]))
    outF.write("\n")
outF.close()
outF = open("./data/validation.article", "w")
for i in range(103,120):
    outF.write(transcript_sep_token1[i])
    outF.write("\n")
outF.close()
outF = open("./data/validation.label", "w")
for i in range(103,120):
    outF.write(str(labels[i]))
    outF.write("\n")
outF.close()
outF = open("./data/test.article", "w")
for i in range(120,137):
    outF.write(transcript_sep_token1[i])
    outF.write("\n")
outF.close()
outF = open("./data/test.label", "w")
for i in range(120,137):
    outF.write(transcript_sep_token1[i])
    outF.write("\n")
outF.close()
with open('labelled_object_without_tokens.pickle', 'wb') as messy_data:
    pickle.dump(dataset_dict, messy_data)

In [ ]:
for i in range(0,len(transcript_sent_tokenized)):
    outF = open("./raw_stories/%s.story" % i, "w")
    for each_sent in transcript_sent_tokenized[i]:
        outF.write(each_sent)
        outF.write("\n\n")
    for each_line in summary_sent_tokenized[i]:
        outF.write("@highlight\n\n")
        outF.write(each_line)
        outF.write("\n\n")
    outF.close()

In [22]:
transcript_sent_tokenized[121]

['all set ?',
 'cool .',
 'alright , it is powerpoint time .',
 "i 've done more powerpoints in this particular experiment than i 've ever done in my life before this experiment which is kind of fun .",
 'so here we have our detailed design meeting where we will look at the prototype and right so , i finally figured out what this whole second bullet point is about in my that my coach was sending to me .',
 "it means i 'm supposed to read the minutes from the previous meeting .",
 'i think .',
 "i do n't know .",
 "otherwise it 's just saying i 'm the secretary and i 'm therefore i 'm taking the minutes , s so just to go just real briefly to go over minutes from last meeting , i will open them slowly , no ?",
 'wait for it , wait for it .',
 'no .',
 "that 's how the wait .",
 'this is , this is very high-powered stuff here , double-clicking , there we go .',
 "so basically the moral of the story from our last minute last meeting was that we that we had meetings from we had presentation

In [23]:
summary_sent_tokenized[121]

["here we have our detailed design meeting i finally figured out what this whole second bullet point is about in my it means i 'm supposed to read the minutes from the previous meeting .",
 "just real briefly to go over minutes from last meeting , basically the moral of the story from our last minute last meeting was that we that we had meetings from we had presentations done by the industrial designer , or from nathan , and ron and from sarah about what we can do here and we essentially came to the conclusion that we should develop a remote with voice recognition , i_e_ that had a vaguely non-remote like shape because you did n't really need to use it as a remote since you could just use your voice .",
 'that would include some mostly just the simple design features for a television operation but with a slide or a fold-out bay for more advanced functions for users .',
 'the u_i_d_ and the i_d_ were asked to go ahead and start developing a prototype for us to look at .',
 'we have asse